# GOOGLE COLAB

In [10]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [157]:
import os
os.chdir('/gdrive/My Drive/Huawei')

In [158]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import datetime

In [64]:
root = '/gdrive/My Drive/Huawei/'
veriler1 = pd.read_csv('film_data.csv')
veriler2 = pd.read_csv('out.csv')


NLP de işlenen descriptionları içeren csv dosyası ile ana csv dosyası birleştirildi.

In [148]:
veriler = pd.concat([veriler1, veriler2], axis=1)
print(veriler.columns)

Index(['Unnamed: 0', 'date_published', 'genre', 'duration', 'country',
       'language', 'description', 'avg_vote', 'votes', '0'],
      dtype='object')


#VERİ ÖNİŞLEME FONKSİYONLARI 

tüm özelliklerin işlendiği yer IMDB_Rate.ipynb dosyasıdır. Elenen veriler ve karşılaştırmalar oradadır.

In [149]:
def datetime_func(veri):
  x = datetime.datetime.now()
  date = veri
  date_list = []
  for i in date:
      y = i.split("-")
      calculate_year = (x.year - int(y[0])) * 365
      calculate_month = (x.month - int(y[1])) * 30
      calculate_day = x.day - int(y[2])
      new_date = calculate_year + calculate_month + calculate_day
      date_list.append(new_date)

  date_df = pd.DataFrame(date_list)
  from sklearn.preprocessing import StandardScaler
  sc = StandardScaler()
  date_df_new = sc.fit_transform(date_df)
  date_df_new = pd.DataFrame(date_df_new, columns=["date_release"], index=veriler.index)
  return date_df_new

In [150]:
def description_func(veri):
  from sklearn import preprocessing
  description = veri

  ohe = preprocessing.OneHotEncoder()
  description = ohe.fit_transform(description).toarray()
  return description

In [151]:
def votes_func(veri):
  from sklearn.preprocessing import StandardScaler
  sc = StandardScaler()

  votes = sc.fit_transform(veri)
  votes = pd.DataFrame(votes, columns=["votes"], index=veriler.index)
  return votes

In [152]:
def genre_func(veri):
  veriler = pd.DataFrame(veri)
  veriler["genre"] = veriler["genre"].str.replace(" ", "")
  veriler["genre"] = veriler["genre"].str.split(",")
  
  mlb = MultiLabelBinarizer()
  genre = veriler['genre']

  encoded = pd.DataFrame(mlb.fit_transform(genre), columns=mlb.classes_, index=veriler.index)

  return encoded

In [153]:
def veri_birlestirme_func(encoded, votes):
  feature1 = pd.DataFrame(data=encoded)
  feature4 = pd.DataFrame(data=votes)
  feature5 = pd.DataFrame(data=duration)
  feature6 = pd.DataFrame(data=date_df_new)
  feature7 = pd.DataFrame(data=description)


  features_new = pd.concat([feature1, feature4], axis=1)
  features_new = pd.concat([features_new, feature5], axis=1)
  features_new = pd.concat([features_new, feature6], axis=1)
  features_new = pd.concat([features_new, feature7], axis=1)
  return features_new


# PREDİCT FONKSİYONU

In [154]:
import pickle
model = pickle.load(open("model.kayit","rb"))
def predict(veriler):
  date_df_new = datetime_func(veriler["date_published"])
  description = description_func(veriler["0"].to_frame())
  duration = veriler["duration"]
  votes = votes_func(veriler[["votes"]])
  encoded= genre_func(veriler["genre"])
  features_new = veri_birlestirme_func(encoded, votes)
  output = model.predict(features_new.values)
  output_df = pd.DataFrame(output, columns=["predicted_rating"], index=veriler.index)
  print(output_df)
  #output_df.to_csv("output.csv")

[19:22:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [155]:
predict(veriler)


       predicted_rating
0              7.419049
1              5.117787
2              6.660292
3              4.684600
4              6.333872
...                 ...
70685          5.070696
70686          6.894261
70687          6.028717
70688          5.554717
70689          7.250593

[70690 rows x 1 columns]


# FARKLI VERİ SETİ GİRİŞİ İÇİN BURAYI KULLANINIZ.

In [159]:
yeni_veri = pd.read_csv('film_data.csv')

In [160]:
import gensim
import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# VERİ ÖNİŞLEME

Karşılaştırmalar ve Veri önişlemelerinin tamamı NLP.ipynb ve IMDB_Rate.ipynb dosyalarında bulunmaktadır.

In [172]:
def veri_okunmasi(data):
  description2 = data["description"]
  nok_isaretleri_kümesi = string.punctuation
  etkisiz_kelimeler_kümesi = ["year","film","0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]
  return description2, nok_isaretleri_kümesi, etkisiz_kelimeler_kümesi

In [171]:
def veri_temizleme(metin, nok_isaretleri_kümesi, etkisiz_kelimeler_kümesi):
    metin = metin.lower()
    metin = metin.replace("\\n", " ")# Verisetimizdeki yeni satır karekterleri, boşluk karekteriyle değiştirdik.
    # Kesme işareti ve sonrasındaki karekterlerin kaldırılması
    metin = re.sub("’(\w+)", "", metin)
    metin = re.sub("'(\w+)", "", metin)
    metin = re.sub("[“,‘,’,”]", "", metin)
    # Sayıların Kaldırılması
    metin = re.sub("[0-9]+", "", metin)
    # Noktalama işaretlerinin kaldırılması
    metin = "".join(list(map(lambda x: x if x not in nok_isaretleri_kümesi else " ", metin)))
    # Etkisiz kelimelerin bir kısmının kaldırılması
    metin = " ".join([i for i in metin.split() if i not in etkisiz_kelimeler_kümesi])
    # Metinde tek kalan harfleri de çıkartalım
    metin = " ".join([i for i in metin.split() if len(i) > 1])

    return metin

In [164]:
def lemma_func(temiz_data):
  from nltk.stem import WordNetLemmatizer
  nltk.download('punkt')
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  lemmatized_data=[]
  for i in temiz_data:
    word_list = nltk.word_tokenize(i)
    lemmatized_data.append(' '.join([lemmatizer.lemmatize(w) for w in word_list]))
  lemmatized_data = pd.Series(lemmatized_data)
  return lemmatized_data

# Daha önce eğitilmiş modelin kullanımı

In [165]:
def lda_func(temiz_data, tokenized_data, kelime_listesi):
  lda_model = gensim.models.ldamodel.LdaModel.load('lda_train.model3')
  class_names = ["0","1","2","3","4","5","6","7","8"]
  son_sonuc = []
  for j in range(len(temiz_data)):
    bow = kelime_listesi.doc2bow(tokenized_data[j])
    results = lda_model.get_document_topics(bow, minimum_probability= 0.0, per_word_topics=False)
    sonuc = []
    for i in range(9):
      sonuc.append(results[i][1])
    son_sonuc.append(class_names[np.argmax([sonuc])])
  return son_sonuc

# Yeni veri setleri için PREDİCT fonksiyonu

In [173]:
def predict_new(veriler):
  description2, nok_isaretleri_kümesi, etkisiz_kelimeler_kümesi = veri_okunmasi(veriler)
  temiz_data = description2.apply(lambda x:veri_temizleme(x, nok_isaretleri_kümesi, etkisiz_kelimeler_kümesi))
  lemmatized_data = lemma_func(temiz_data)
  tokenized_data = lemmatized_data.apply(lambda x: x.split())
  kelime_listesi = gensim.corpora.Dictionary(tokenized_data)
  dokuman_terim_matrisi = [kelime_listesi.doc2bow(terim) for terim in tokenized_data]
  son_sonuc = lda_func(temiz_data, tokenized_data, kelime_listesi)
  son_sonuc = pd.DataFrame(son_sonuc)

  date_df_new = datetime_func(veriler["date_published"])
  description = description_func(son_sonuc)
  duration = veriler["duration"]
  votes = votes_func(veriler[["votes"]])
  encoded= genre_func(veriler["genre"])
  features_new = veri_birlestirme_func(encoded, votes)
  output = model.predict(features_new.values)
  output_df = pd.DataFrame(output, columns=["predicted_rating"], index=veriler.index)
  print(output_df)

In [174]:
predict_new(yeni_veri)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
       predicted_rating
0              7.419049
1              5.117787
2              6.660292
3              4.684600
4              6.333872
...                 ...
70685          5.070696
70686          6.894261
70687          6.028717
70688          5.554717
70689          7.250593

[70690 rows x 1 columns]
